## 기본설정 및 함수정의

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 영등포역 위도, 경도
ydp = [37.51569886583137, 126.90768824079629]

# Point를만드는 함수
def make_point(x):
    try:
        return Point(x)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

#### 영등포구 행정경계 (출처 - 통계지리정보서비스 2023년 센서스용 행정구역경계(읍면동))

In [2]:
# shp to geodataframe convert
shapefile_path = "SBJ_2406_001/_census_data_2023_bnd_dong_bnd_dong_11190_2023_2023\\bnd_dong_11190_2023_2023_2Q.shp"
ydp_gdf = gpd.read_file(shapefile_path)
ydp_gdf = ydp_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

#### 격자(매핑용)

In [3]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/19.영등포_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_map_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

#### 영등포구 상세 도로망

In [4]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/6.영등포_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
roadsystem_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

#### 도로명주소(도로)

In [5]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/7.영등포_도로명주소(도로).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_df = pd.json_normalize(geojson_data['features'])
roadname_df['geometry.coordinates'] = roadname_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadname_df.rename(columns={'geometry.coordinates' : 'geometry'}, inplace=True)
# roadname_df 데이터프레임을 GeoDataFrame으로 변환
roadname_df = gpd.GeoDataFrame(roadname_df, geometry='geometry')

#### 소방용수시설 시각화

In [6]:
# 소방용수시설 데이터프레임을 GeoPandas 데이터프레임으로 변환
firefighting_water_df = pd.read_csv('SBJ_2406_001/22.영등포_소방용수시설.csv')

#### 소방서 위치정보

In [7]:
# 소방서 데이터프레임을 GeoPandas 데이터프레임으로 변환
firehouse_df = pd.read_csv('SBJ_2406_001/23.영등포_소방서위치정보.csv')

In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_firefacility_map = KeplerGl(height=1000, width=1500)
ydp_firefacility_map.add_data(data=firehouse_df, name="영등포구 소방서 데이터")
ydp_firefacility_map.add_data(data=firefighting_water_df, name="영등포구 소방용수시설 데이터")
ydp_firefacility_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_firefacility_map.add_data(data=roadname_df, name="영등포구 도로망 데이터")

# 맵 출력 및 상세설정
ydp_firefacility_map

In [30]:
# 맵 저장
ydp_firefacility_map.save_to_html(file_name="visualization/영등포구 건물 및 소방관련/영등포구 소방시설 시각화 map.html")

Map saved to visualization/영등포구 건물 및 소방관련/영등포구 소방시설 시각화 map.html!


#### 건물노후도 정보

In [11]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/12.영등포_건물노후도.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
building_oldness_df = pd.json_normalize(geojson_data['features'])
building_oldness_df['geometry.coordinates'] = building_oldness_df['geometry.coordinates'].apply(lambda x : make_pol(x))
building_oldness_df.rename(columns={'geometry.coordinates' : 'geometry'}, inplace=True)
# building_oldness_df 데이터프레임을 GeoDataFrame으로 변환
building_oldness_df = gpd.GeoDataFrame(building_oldness_df, geometry='geometry')
building_oldness_df['properties.old_year'] = building_oldness_df['properties.old_year'].astype(float)
building_oldness_df = building_oldness_df[['properties.emd_nm', 'properties.strct_nm', 'properties.usage_nm', 'properties.old_year', 'geometry']]

In [12]:
# 노후 및 불량건축물 비율계산
old_building = building_oldness_df[building_oldness_df['properties.old_year'] >= 20]
old_building_rate = round((len(old_building)/len(building_oldness_df)) * 100, 2)
old_building_rate

81.2

In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_building_oldness_map = KeplerGl(height=1000, width=1500)
ydp_building_oldness_map.add_data(data=building_oldness_df, name="영등포구 건물노후도 데이터")
ydp_building_oldness_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_building_oldness_map.add_data(data=roadname_df, name="영등포구 도로망 데이터")

# 맵 출력 및 상세설정
ydp_building_oldness_map

In [70]:
# 맵 저장
ydp_building_oldness_map.save_to_html(file_name="visualization/영등포구 건물 및 소방관련/영등포구 건물노후도 시각화 map.html")

Map saved to visualization/영등포구 건물 및 소방관련/영등포구 건물노후도 시각화 map.html!


#### 건물 시각화

In [13]:
# 표제부 데이터프레임 로드 후 지번 - 건물구조, 지번 - 건물용도 dict생성
pyo_df = pd.read_csv('SBJ_2406_001/14.영등포_건축물대장_표제부.csv')
pyo_df['plot_lctn'] = pyo_df['plot_lctn'].apply(lambda x : x.split(" 영등포구 ")[-1])
pyo_df['plot_lctn'] = pyo_df['plot_lctn'].apply(lambda x : x.strip("번지"))
jibun_strct_dic = dict(zip(pyo_df['plot_lctn'], pyo_df['strct_nm']))
jibun_use_dic = dict(zip(pyo_df['plot_lctn'], pyo_df['use_nm']))

# 지번 - 법정동 코드 dict 생성
pyo_df['plot_lctn'] = pyo_df['plot_lctn'].apply(lambda x : x.split(" ")[0])
pyo_df['emd_cd'] = pyo_df['emd_cd'].astype(str)
pyo_df['emd_cd'] = pyo_df['emd_cd'].apply(lambda x : x[:3])
jibun_emd_dic = dict(zip(pyo_df['emd_cd'], pyo_df['plot_lctn']))

# GeoJSON 파일 불러오기
with open('SBJ_2406_001/11.영등포_도로명주소(건물).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
rbuilding_df = pd.json_normalize(geojson_data['features'])
rbuilding_df['geometry.coordinates'] = rbuilding_df['geometry.coordinates'].apply(lambda x : make_pol(x))
rbuilding_df.rename(columns={'geometry.coordinates' : 'geometry'}, inplace=True)
rbuilding_df.rename(columns={'properties.GRO_FLO_CO' : 'GRO_FLO_CO'}, inplace=True)
rbuilding_df.rename(columns={'properties.LNBR_MNNM' : 'LNBR_MNNM'}, inplace=True)
rbuilding_df.rename(columns={'properties.LNBR_SLNO' : 'LNBR_SLNO'}, inplace=True)
# rbuilding_df 데이터프레임을 GeoDataFrame으로 변환
rbuilding_df = gpd.GeoDataFrame(rbuilding_df, geometry='geometry')
rbuilding_df['properties.EMD_CD'] = rbuilding_df['properties.EMD_CD'].astype(str)
rbuilding_df['LNBR_MNNM'] = rbuilding_df['LNBR_MNNM'].astype(str)
rbuilding_df['LNBR_SLNO'] = rbuilding_df['LNBR_SLNO'].astype(str)
rbuilding_df['EMD_NM'] = rbuilding_df['properties.EMD_CD'].map(jibun_emd_dic)

# plot_lctn 열 생성 함수 정의
def create_plot_lctn(row):
    if row['LNBR_SLNO'] == '0':
        return f"{row['EMD_NM']} {row['LNBR_MNNM']}"
    else:
        return f"{row['EMD_NM']} {row['LNBR_MNNM']}-{row['LNBR_SLNO']}"
# 새로운 열 추가
rbuilding_df['plot_lctn'] = rbuilding_df.apply(create_plot_lctn, axis=1)
rbuilding_df['strct'] = rbuilding_df['plot_lctn'].map(jibun_strct_dic)
rbuilding_df['use'] = rbuilding_df['plot_lctn'].map(jibun_use_dic)
rbuilding_df = rbuilding_df[['plot_lctn', 'strct', 'use', 'GRO_FLO_CO', 'geometry']]

In [ ]:
# 맵 객체 생성 및 데이터 로드
rbuilding_df_n = rbuilding_df.dropna()
ydp_building_map = KeplerGl(height=1000, width=1500)
ydp_building_map.add_data(data=rbuilding_df_n, name="영등포구 건물 데이터")
ydp_building_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_building_map.add_data(data=roadname_df, name="영등포구 도로망 데이터")

# 맵 출력 및 상세설정
ydp_building_map

In [267]:
# 맵 저장
ydp_building_map.save_to_html(file_name="visualization/영등포구 건물 및 소방관련/영등포구 건물 시각화 map.html")

Map saved to visualization/영등포구 건물 및 소방관련/영등포구 건물 시각화 map.html!


#### 건물에너지(전기 + 가스) 시각화

In [ ]:
# 건물에너지 데이터프레임을 GeoPandas 데이터프레임으로 변환
electric_power_df = pd.read_csv('SBJ_2406_001/16.영등포_건물에너지(전기).csv')
electric_power_df['plot_lctn'] = electric_power_df['plot_lctn'].apply(lambda x : x.split(" 영등포구 ")[-1])
electric_power_df['plot_lctn'] = electric_power_df['plot_lctn'].apply(lambda x : x.strip("번지"))
electric_power_df = electric_power_df[['plot_lctn', 'usage', 'use_ym']]
electric_power_df['use_ym'] = electric_power_df['use_ym'].astype(str)
electric_power_df['use_ym'] = pd.to_datetime(electric_power_df['use_ym'], format='%Y%m')
electric_power_df = electric_power_df.sort_values(by='use_ym')
electric_power_df['use_ym'] = electric_power_df['use_ym'].astype(str)

gas_power_df = pd.read_csv('SBJ_2406_001/17.영등포_건물에너지(가스).csv')
gas_power_df['plot_lctn'] = gas_power_df['plot_lctn'].apply(lambda x : x.split(" 영등포구 ")[-1])
gas_power_df['plot_lctn'] = gas_power_df['plot_lctn'].apply(lambda x : x.strip("번지"))
gas_power_df = gas_power_df[['plot_lctn', 'usage', 'use_ym']]
gas_power_df['use_ym'] = gas_power_df['use_ym'].astype(str)
gas_power_df['use_ym'] = pd.to_datetime(gas_power_df['use_ym'], format='%Y%m')
gas_power_df = gas_power_df.sort_values(by='use_ym')
gas_power_df['use_ym'] = gas_power_df['use_ym'].astype(str)

In [90]:
# 건물 : polygon 구조를 가지는 dict생성(mapping 목적)
building_polygon_dic = dict(zip(rbuilding_df['plot_lctn'], rbuilding_df['geometry']))
rbuilding_df_energy = rbuilding_df.copy()
# 지번별 누적 전기사용량 계산
cumulative_e_usage = electric_power_df.groupby('plot_lctn')['usage'].sum().reset_index()
cumulative_e_usage_dic = dict(zip(cumulative_e_usage['plot_lctn'], cumulative_e_usage['usage']))
rbuilding_df_energy['cumulative_e_usage'] = rbuilding_df_energy['plot_lctn'].map(cumulative_e_usage_dic)

# 지번별 평균 전기사용량 계산
average_e_usage = electric_power_df.groupby('plot_lctn')['usage'].mean().reset_index()
average_e_usage_dic = dict(zip(average_e_usage['plot_lctn'], average_e_usage['usage']))
rbuilding_df_energy['average_e_usage'] = rbuilding_df_energy['plot_lctn'].map(average_e_usage_dic)

# 지번별 누적 가스사용량 계산
cumulative_g_usage = gas_power_df.groupby('plot_lctn')['usage'].sum().reset_index()
cumulative_g_usage_dic = dict(zip(cumulative_g_usage['plot_lctn'], cumulative_g_usage['usage']))
rbuilding_df_energy['cumulative_g_usage'] = rbuilding_df_energy['plot_lctn'].map(cumulative_g_usage_dic)
# 월별 평균 가스사용량 계산
average_g_usage = gas_power_df.groupby('plot_lctn')['usage'].mean().reset_index()
average_g_usage_dic = dict(zip(average_g_usage['plot_lctn'], average_g_usage['usage']))
rbuilding_df_energy['average_g_usage'] = rbuilding_df_energy['plot_lctn'].map(average_g_usage_dic)

# 결측값 필터링
columns_to_check_e = ['cumulative_e_usage', 'average_e_usage']
columns_to_check_g = ['cumulative_g_usage','average_g_usage']
rbuilding_df_energy_e = rbuilding_df_energy.dropna(subset=columns_to_check_e)
rbuilding_df_energy_g = rbuilding_df_energy.dropna(subset=columns_to_check_g)

In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_energy_usage_map = KeplerGl(height=1000, width=1500)
ydp_energy_usage_map.add_data(data=rbuilding_df_energy_e, name="영등포구 전기에너지사용 데이터")
ydp_energy_usage_map.add_data(data=rbuilding_df_energy_g, name="영등포구 가스에너지사용 데이터")
ydp_energy_usage_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_energy_usage_map.add_data(data=roadname_df, name="영등포구 도로망 데이터")

# 맵 출력 및 상세설정
ydp_energy_usage_map

In [94]:
# 맵 저장
ydp_energy_usage_map.save_to_html(file_name="visualization/영등포구 건물 및 소방관련/영등포구 건물 에너지사용 시각화 map.html")

Map saved to visualization/영등포구 건물 및 소방관련/영등포구 건물 에너지사용 시각화 map.html!
